In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip install --quiet datasets
!pip install --quiet qatch
!pip install --quiet transformers[torch]

Cloning into 'transformers'...
remote: Enumerating objects: 216124, done.
remote: Counting objects: 100% (1005/1005), done.
remote: Compressing objects: 100% (635/635), done.
remote: Total 216124 (delta 516), reused 655 (delta 298), pack-reused 215119
Receiving objects: 100% (216124/216124), 227.40 MiB | 12.10 MiB/s, done.
Resolving deltas: 100% (156178/156178), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source o

In [ ]:
!pip install --quiet transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.7 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import drive
import pickle
import pandas as pd
import logging
import warnings
from qatch import MetricEvaluator
from qatch.database_reader import MultipleDatabases
import torch

In [ ]:
os.chdir("/content/transformers/examples/research_projects/tapex")##only for training
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title get_sql_tag
def get_sql_query_level(sql_query):
    # Function to classify SQL query and return just the level tag
    def classify_sql_query_level(sql_query):
        sql_query_lower = sql_query.lower()
        level = 6  # Default to OTHER

        if "select" in sql_query_lower and "from" in sql_query_lower:
            level = 0  # SELECT
            if "max(" in sql_query_lower or "min(" in sql_query_lower or \
               "avg(" in sql_query_lower or "count(" in sql_query_lower or \
               "sum(" in sql_query_lower:
                level = 1  # SIMPLE_AGGR

        if "where" in sql_query_lower:
            level = 2  # WHERE

        if "group by" in sql_query_lower:
            level = 3  # GROUPBY

        if "having" in sql_query_lower:
            level = 4  # HAVING

        if "order by" in sql_query_lower:
            level = 5  # ORDERBY

        return level

    # Classification tags
    classification_tags = {
        0: "SELECT",
        1: "SIMPLE_AGGR",
        2: "WHERE",
        3: "GROUPBY",
        4: "HAVING",
        5: "ORDERBY",
        6: "OTHER"
    }

    # Classify the SQL query
    level = classify_sql_query_level(sql_query)

    # Return the classification tag
    return classification_tags[level]

In [ ]:
with open('/content/drive/MyDrive/spider/db_table_to_df_valid.pkl', 'rb') as file:
    db_to_df_spiderDev = pickle.load(file)

In [ ]:
spider_test_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final_valid.pkl')
spider_test_pd['sql_tags'] = spider_test_pd['query'].apply(get_sql_query_level)

In [ ]:
db_save_path = '/content/drive/MyDrive/ProprietaryDatasets/database4'
databases = MultipleDatabases(db_save_path)
evaluator = MetricEvaluator(databases=databases)

In [ ]:
#For spider Dev
db_save_path_spiderDev = '/content/drive/MyDrive/spider/test_database'
databasesSpider_Dev = MultipleDatabases(db_save_path_spiderDev)
evaluator = MetricEvaluator(databases=databasesSpider_Dev)

In [ ]:
with open('/content/drive/MyDrive/ProprietaryDatasetsDict.pkl', 'rb') as f:
      db_to_df = pickle.load(f)
qatch_pickle = pd.read_pickle('/content/drive/MyDrive/spider/QATCH_proprietary_v2.pkl')
ProprietaryDomains=['medicine','miscellaneous','ecommerce','finance']
qatch_pickle=qatch_pickle[qatch_pickle['domain']==ProprietaryDomains[0]]
qatch_pickle['sql_tags'] = qatch_pickle['sql_tags'].apply(lambda x: x.split('-')[0])

In [ ]:
percentage_valid = 0.2
temp_indices = qatch_pickle.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_valid, random_state=42)).index
temp_df = qatch_pickle.loc[temp_indices]
train_df_full = qatch_pickle.drop(temp_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {qatch_pickle.shape[0]}")
print(f"Training set size: {train_df_full.shape[0]}")
print(f"Tempset size: {temp_df.shape[0]}")

Total dataset size: 429
Training set size: 344
Tempset size: 85


In [ ]:
percentage_test = 0.5
test_indices = temp_df.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_test, random_state=42)).index
test_df = qatch_pickle.loc[test_indices]
valid_df = temp_df.drop(test_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {temp_df.shape[0]}")
print(f"Valid set size: {valid_df.shape[0]}")
print(f"Test size: {test_df.shape[0]}")

Total dataset size: 85
Valid set size: 42
Test size: 43


In [ ]:
#Now lets sample from teh trainset as we are not going to use all of it
percentage_train = 0.1
train_indices = train_df_full.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_train, random_state=42)).index
train_df = train_df_full.loc[train_indices]

# Check the size of each set to confirm the split
print(f"Total dataset size: {train_df_full.shape[0]}")
print(f"Training set size: {train_df.shape[0]}")

Total dataset size: 344
Training set size: 34


In [ ]:
#@title run_model_wikisql
%%writefile my_script.py
#!/usr/bin/env python
# coding=utf-8
# Copyright 2022 The Microsoft and The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Fine-tuning the library models for tapex on table-based question answering tasks.
Adapted from script: https://github.com/huggingface/transformers/blob/master/examples/pytorch/summarization/run_summarization.py
"""

import logging
import os
import sys
from collections import defaultdict
from dataclasses import dataclass, field
from functools import partial
from typing import List, Optional

import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
import pandas as pd
from datasets import load_dataset
from filelock import FileLock

import transformers
from transformers import (
    AutoConfig,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TapexTokenizer,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.utils import check_min_version


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.17.0.dev0")

logger = logging.getLogger(__name__)

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"},
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Pretrained tokenizer name or path if not the same as model_name. "
                "By default we use BART-large tokenizer for TAPEX-large."
            )
        },
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": (
                "Will use the token generated when running `huggingface-cli login` (necessary to use this script "
                "with private models)."
            )
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines or csv file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "An optional input evaluation data file to evaluate the metrics (rouge) on (a jsonlines or csv file)."
            )
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input test data file to evaluate the metrics (rouge) on (a jsonlines or csv file)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": (
                "The maximum total sequence length for target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "The maximum total sequence length for validation target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`. "
                "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
                "during ``evaluate`` and ``predict``."
            )
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether to pad all samples to model maximum sentence length. "
                "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
                "efficient on GPU but very bad for TPU."
            )
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of prediction examples to this "
                "value if set."
            )
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
                "which is used during ``evaluate`` and ``predict``."
            )
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    # Detecting last checkpoint.
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )
        elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
            logger.info(
                f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
                "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
            )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )
    logger.setLevel(logging.INFO if is_main_process(training_args.local_rank) else logging.WARN)

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    # Set the verbosity to info of the Transformers logger (on main process only):
    if is_main_process(training_args.local_rank):
        transformers.utils.logging.set_verbosity_info()
    logger.info(f"Training/evaluation parameters {training_args}")

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Get the datasets: you can either provide your own CSV/JSON training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For JSON files, this script will use the `question` column for the input question and `table` column for the corresponding table.
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        datasets = load_dataset(data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir)
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
            extension = data_args.train_file.split(".")[-1]
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
            extension = data_args.validation_file.split(".")[-1]
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
            extension = data_args.test_file.split(".")[-1]
        datasets = load_dataset(extension, data_files=data_files, cache_dir=model_args.cache_dir)

    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        token=True if model_args.use_auth_token else None,
    )

    # IMPORTANT: the initial BART model's decoding is penalized by no_repeat_ngram_size, and thus
    # we should disable it here to avoid problematic generation
    config.no_repeat_ngram_size = 0
    config.max_length = 1024
    config.early_stopping = False

    # load tapex tokenizer
    tokenizer = TapexTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        revision=model_args.model_revision,
        token=True if model_args.use_auth_token else None,
        add_prefix_space=True,
    )

    # load Bart based Tapex model (default tapex-large)
    model = BartForConditionalGeneration.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        token=True if model_args.use_auth_token else None,
    )

    if model.config.decoder_start_token_id is None:
        raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

    # Preprocessing the datasets.
    # We need to tokenize inputs and targets.
    if training_args.do_train:
        column_names = datasets["train"].column_names
    elif training_args.do_eval:
        column_names = datasets["validation"].column_names
    elif training_args.do_predict:
        column_names = datasets["test"].column_names
    else:
        logger.info("There is nothing to do. Please pass `do_train`, `do_eval` and/or `do_predict`.")
        return

    # Temporarily set max_target_length for training.
    max_target_length = data_args.max_target_length
    padding = "max_length" if data_args.pad_to_max_length else False

    if training_args.label_smoothing_factor > 0 and not hasattr(model, "prepare_decoder_input_ids_from_labels"):
        logger.warning(
            "label_smoothing is enabled but the `prepare_decoder_input_ids_from_labels` method is not defined for "
            f"`{model.__class__.__name__}`. This will lead to loss being calculated twice and will take up more memory"
        )

    def preprocess_tableqa_function(examples, is_training=False):
        """
        The is_training FLAG is used to identify if we could use the supervision
        to truncate the table content if it is required.
        """

        questions = [question.lower() for question in examples["question"]]
        example_tables = examples["table"]
        tables = [
            pd.DataFrame.from_records(example_table["rows"], columns=example_table["header"])
            for example_table in example_tables
        ]

        # using wikitablequestion's answer set
        answers = examples["answers"]

        # IMPORTANT: we cannot pass by answers during evaluation, answers passed during training are used to
        # truncate large tables in the train set!
        if is_training:
            model_inputs = tokenizer(
                table=tables,
                query=questions,
                answer=answers,
                max_length=data_args.max_source_length,
                padding=padding,
                truncation=True,
            )
        else:
            model_inputs = tokenizer(
                table=tables, query=questions, max_length=data_args.max_source_length, padding=padding, truncation=True
            )

        labels = tokenizer(
            answer=[", ".join(answer) for answer in answers],
            max_length=max_target_length,
            padding=padding,
            truncation=True,
        )

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length" and data_args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]

        return model_inputs

    # in training, we can use the answer as extra information to truncate large tables
    preprocess_tableqa_function_training = partial(preprocess_tableqa_function, is_training=True)

    if training_args.do_train:
        if "train" not in datasets:
            raise ValueError("--do_train requires a train dataset")
        train_dataset = datasets["train"]
        if data_args.max_train_samples is not None:
            train_dataset = train_dataset.select(range(data_args.max_train_samples))
        train_dataset = train_dataset.map(
            preprocess_tableqa_function_training,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
        )

    if training_args.do_eval:
        max_target_length = data_args.val_max_target_length
        if "validation" not in datasets:
            raise ValueError("--do_eval requires a validation dataset")
        eval_dataset = datasets["validation"]
        if data_args.max_eval_samples is not None:
            eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))
        eval_dataset = eval_dataset.map(
            preprocess_tableqa_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
        )

    if training_args.do_predict:
        max_target_length = data_args.val_max_target_length
        if "test" not in datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_dataset = datasets["test"]
        if data_args.max_predict_samples is not None:
            predict_dataset = predict_dataset.select(range(data_args.max_predict_samples))
        predict_dataset = predict_dataset.map(
            preprocess_tableqa_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
        )

    # Data collator
    label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8 if training_args.fp16 else None,
    )

    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        if data_args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        delimiter = ", "

        # define example evaluation
        def evaluate_example(predict_str: str, ground_str: str):
            predict_spans = predict_str.split(delimiter)
            ground_spans = ground_str.split(delimiter)
            predict_values = defaultdict(lambda: 0)
            ground_values = defaultdict(lambda: 0)
            for span in predict_spans:
                try:
                    predict_values[float(span)] += 1
                except ValueError:
                    predict_values[span.strip()] += 1
            for span in ground_spans:
                try:
                    ground_values[float(span)] += 1
                except ValueError:
                    ground_values[span.strip()] += 1
            _is_correct = predict_values == ground_values
            return _is_correct

        def get_denotation_accuracy(predictions: List[str], references: List[str]):
            assert len(predictions) == len(references)
            correct_num = 0
            for predict_str, ground_str in zip(predictions, references):
                is_correct = evaluate_example(predict_str.lower(), ground_str.lower())
                if is_correct:
                    correct_num += 1
            return correct_num / len(predictions)

        accuracy = get_denotation_accuracy(decoded_preds, decoded_labels)
        result = {"denotation_accuracy": accuracy}

        return result

    # Initialize our Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if training_args.predict_with_generate else None,
    )

    if training_args.do_train:
        checkpoint = None
        if training_args.resume_from_checkpoint is not None:
            checkpoint = training_args.resume_from_checkpoint
        elif last_checkpoint is not None:
            checkpoint = last_checkpoint
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        trainer.save_model()  # Saves the tokenizer too for easy upload

        metrics = train_result.metrics
        max_train_samples = (
            data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
        )
        metrics["train_samples"] = min(max_train_samples, len(train_dataset))

        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()

    # Evaluation
    results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        metrics = trainer.evaluate(
            max_length=data_args.val_max_target_length, num_beams=data_args.num_beams, metric_key_prefix="eval"
        )
        max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
        metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    if training_args.do_predict:
        logger.info("*** Predict ***")

        predict_results = trainer.predict(
            predict_dataset,
            metric_key_prefix="predict",
            max_length=data_args.val_max_target_length,
            num_beams=data_args.num_beams,
        )
        metrics = predict_results.metrics
        max_predict_samples = (
            data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
        )
        metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

        trainer.log_metrics("predict", metrics)
        trainer.save_metrics("predict", metrics)

        if trainer.is_world_process_zero():
            if training_args.predict_with_generate:
                predictions = tokenizer.batch_decode(
                    predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
                predictions = [pred.strip() for pred in predictions]
                output_prediction_file = os.path.join(training_args.output_dir, "tapex_predictions.txt")
                with open(output_prediction_file, "w") as writer:
                    writer.write("\n".join(predictions))

    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()

Writing my_script.py


In [ ]:
  # --max_steps 600 \Remove this and put 5 epochs

In [ ]:
!python my_script.py \
  --do_train \
  --do_eval \
  --output_dir /content/drive/MyDrive/TAPEX_Ressources/checkpoints/train_df_spider_100prct_checkpoint/ \
  --model_name_or_path microsoft/tapex-base \
  --overwrite_output_dir \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 1 \
  --per_device_eval_batch_size 8 \
  --learning_rate 3e-5 \
  --logging_steps 10 \
  --eval_steps 200 \
  --save_steps 200 \
  --warmup_steps 200 \
  --evaluation_strategy steps \
  --predict_with_generate \
  --num_beams 5 \
  --weight_decay 1e-2 \
  --label_smoothing_factor 0.1 \
  --num_train_epochs=5 \
  --train_file /content/drive/MyDrive/TAPEX_Ressources/QATCH_DATA_TAPEX/train_df_spider_100prct.json \
  --validation_file /content/drive/MyDrive/TAPEX_Ressources/QATCH_DATA_TAPEX/valid_df_spider_10prct.json

2024-06-19 11:42:10.437602: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 11:42:10.437656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 11:42:10.445272: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 11:42:10.466747: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 11:42:12.752399: W tensorflow/compiler/tf2

In [ ]:
#@title model class
##Modified code from qatch repo
import torch
import pandas as pd
from collections import Counter, defaultdict
import numpy as np
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
import warnings

class MyModel:
    def __init__(self, model_name_or_path, device):
        config = AutoConfig.from_pretrained(model_name_or_path)
        tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, config=config)

        self.config = config
        self.tokenizer = tokenizer
        self.model = model
        self.device = device

    def process_input(self, table: pd.DataFrame,
                          query: str,
                          tbl_name: str):
            if table.shape[0] * table.shape[1] > 1024:
                return None

            # convert table to string
            table = table.astype(str)
            # process table
            for col in table.columns:
                table[col] = table[col].str.lower()

            # tapex accepts uncased input since it is pre-trained on the uncased corpus
            query = query.lower()
            try:
                model_input = self.tokenizer(table=table, query=query,
                                            padding=True, return_tensors="pt")
            except ValueError as e:
                # we get error when the tokenized input is longer than accepted from model
                logging.warning(e)
                return None

            if model_input.input_ids.shape[1] > 1024:
                warnings.warn(f'After tokenization'
                              f' the input is longer than 1024 tokens: '
                              f'{model_input.input_ids.shape[1]}. '
                              'the input will be skipped')
                return None

            return model_input.to(self.device)

    def predict_input(self, model_input, table):
#I added this ,max_new_tokens=1024 to spress a warning during inference on spiderDev
            outputs = self.model.generate(**model_input,max_new_tokens=1024)
            model_input.to('cpu')

            [model_input[idx].detach() for idx in model_input]
            outputs = outputs.detach().cpu()

            # decode back to text
            pred_cells_queries = self.tokenizer.batch_decode(outputs,
                                                            skip_special_tokens=True)
            # the output contains list of string for each query. Manually transform the output
            answers = []
            for pred_query in pred_cells_queries:
                query_ans = self._return_cells_aggr_by_row(table, pred_query)
                answers.extend(query_ans)
            del model_input
            del outputs
            return answers

    @staticmethod
    def _return_cells_aggr_by_row(table, pred_query):
            """
            Perform an aggregation operation by row of the cells in a specified table based on a predicate query.

            Args:
                table (np.array): The table to perform the operation on.
                pred_query (str): The predicate query used for aggregation operation. It should be a string of comma
                                  separated cell values, e.g., "cell1,cell2,cell3".

            Returns:
                list: Returns a list of lists where each sublist contains aggregated cell values from a single row of the table.

            Example:
                Let's assume we have a table as below:

                [["cell1", "cell2"],
                ["cell3", "cell1"],
                ["cell1", "cell2"]]

                And pred_query as "cell1,cell1,cell2"

                Calling _return_cells_aggr_by_row(table, pred_query) will give:

                [["cell1", "cell2"], ["cell1"], ["cell1", "cell2"]]

            Note:
                If a cell from the pred_query is not present in the table, the method treats it as if it's in an imaginary
                row indexed as -1. Therefore, if you see a [-1] in the result, it means one or more cells in your pred_query
                did not appear in the table.
            """
            # Initializing a defaultdict to store the results of the query
            query_ans = defaultdict(list)
            # Splitting the query into cells
            cells: list = pred_query.split(",")
            # Counting the occurrences of each cell in the query
            counted_cells = Counter(cells)
            # Iterating over each cell type and its count from the counted_cells
            for cell, count in counted_cells.items():
                # Finding the row ids where the current cell type exists in the table
                row_ids = np.where(table == cell.strip())[0]
                if len(row_ids) == 0:
                    # If the cell is not present in the table, set the row_id as -1
                    row_ids = [-1]

                # If the count of cell in the query > 1, select the first 'count' number of rows
                if count > 1:
                    row_ids = row_ids[:count]
                # Appending the cell to the rows in the query_ans for each row_id
                [query_ans[idx].append(cell.strip()) for idx in row_ids]
            # Return the aggregated cells from each row as a list of lists
            return list(query_ans.values())

In [ ]:
# Load the model and tokenizer
model_name_or_path = "/content/drive/MyDrive/TAPEX_Ressources/checkpoints/train_df_spider_100prct_checkpoint/"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
my_model = MyModel(model_name_or_path, device)
my_model.model.to(device)
my_model.model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
#@title tapas base no training
# Load the model and tokenizer
model_name_or_path = "microsoft/tapex-base"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
my_model = MyModel(model_name_or_path, device)
my_model.model.to(device)
my_model.model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [ ]:
def getAnswer(row):
    row_index = row['ID']
    table_used = row['table_used']
    table = db_to_df[table_used]
    table = table.astype(str)
    query_nl = row["question"]

    # Preprocess the input data and make predictions
    model_input = my_model.process_input(table, query_nl, table_used)
    if model_input is not None:
        answers = my_model.predict_input(model_input, table)
        return answers
    else:
        print(f"The input for row {row_index} was too long and was skipped.")
        return None

def getAnswerSpiderDev(row):
    row_index = row['ID']

    seq_id = row["db_id"] + "_X_" + row["table_used"]
    if seq_id not in db_to_df_spiderDev or db_to_df_spiderDev[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None

    table_used=row["table_used"]
    table = db_to_df_spiderDev[seq_id]
    table = table.astype(str)
    query_nl = row["question"]
    # Preprocess the input data and make predictions
    model_input = my_model.process_input(table, query_nl, table_used)
    if model_input is not None:
        answers = my_model.predict_input(model_input, table)
        return answers
    else:
        print(f"The input for row {row_index} was too long and was skipped.")
        return None



In [ ]:
#@title calculate_and_log_stats
def calculate_and_log_stats(epoch,avg_loss,df,valid=True):
    my_model.model.eval()
    with torch.no_grad():
      df_cp=df.copy()
      df_cp['predictions_TAPAS'] = df_cp.apply(getAnswer, axis=1)
      tests_df_results = evaluator.evaluate_with_df(df_cp,
                                            prediction_col_name=f'predictions_TAPAS',
                                            task="QA")
    my_model.model.train()

    columns_to_average = [
        'cell_precision_predictions_TAPAS',
        'cell_recall_predictions_TAPAS',
        'tuple_cardinality_predictions_TAPAS',
        'tuple_constraint_predictions_TAPAS',
        'tuple_order_predictions_TAPAS'
    ]


    general_stats = {
        'epoch': epoch,
        'avg_loss': avg_loss,
        'general_avg_cell_precision': tests_df_results[columns_to_average[0]].mean(),
        'general_avg_cell_recall': tests_df_results[columns_to_average[1]].mean(),
        'general_avg_tuple_cardinality': tests_df_results[columns_to_average[2]].mean(),
        'general_avg_tuple_constraint': tests_df_results[columns_to_average[3]].mean()
    }

    print(f"General Model Performance Metrics for epoch:{epoch}")
    for stat_name, stat_value in general_stats.items():
        if stat_name != 'epoch':  # We don't need to print the epoch again
            print(f"{stat_name}: {stat_value}")
    print("----------------------------")

    grouped_means = tests_df_results.groupby('sql_tags')[columns_to_average].mean()


    for sql_tag, row in grouped_means.iterrows():
        for col in columns_to_average:
            key_name = f"{sql_tag}_{col}"
            general_stats[key_name] = row[col]

    print("Grouped Stats by SQL Tag:")
    for sql_tag, metrics in grouped_means.iterrows():
        print(f"SQL Tag: {sql_tag}")
        for col in columns_to_average:
            print(f"  {col}: {metrics[col]}")
        print("----------------------------")
    return general_stats


In [ ]:
#@title calculate_and_log_stats for Spider
def calculate_and_log_stats(epoch,avg_loss,df,valid=True):
    my_model.model.eval()
    with torch.no_grad():
      df_cp=df.copy()
      df_cp['predictions_TAPAS'] = df_cp.apply(getAnswerSpiderDev, axis=1)
      tests_df_results = evaluator.evaluate_with_df(df_cp,
                                            prediction_col_name=f'predictions_TAPAS',
                                            task="QA")
    my_model.model.train()

    columns_to_average = [
        'cell_precision_predictions_TAPAS',
        'cell_recall_predictions_TAPAS',
        'tuple_cardinality_predictions_TAPAS',
        'tuple_constraint_predictions_TAPAS',
        'tuple_order_predictions_TAPAS'
    ]


    general_stats = {
        'epoch': epoch,
        'avg_loss': avg_loss,
        'general_avg_cell_precision': tests_df_results[columns_to_average[0]].mean(),
        'general_avg_cell_recall': tests_df_results[columns_to_average[1]].mean(),
        'general_avg_tuple_cardinality': tests_df_results[columns_to_average[2]].mean(),
        'general_avg_tuple_constraint': tests_df_results[columns_to_average[3]].mean()
    }

    print(f"General Model Performance Metrics for epoch:{epoch}")
    for stat_name, stat_value in general_stats.items():
        if stat_name != 'epoch':  # We don't need to print the epoch again
            print(f"{stat_name}: {stat_value}")
    print("----------------------------")

    grouped_means = tests_df_results.groupby('sql_tags')[columns_to_average].mean()


    for sql_tag, row in grouped_means.iterrows():
        for col in columns_to_average:
            key_name = f"{sql_tag}_{col}"
            general_stats[key_name] = row[col]

    print("Grouped Stats by SQL Tag:")
    for sql_tag, metrics in grouped_means.iterrows():
        print(f"SQL Tag: {sql_tag}")
        for col in columns_to_average:
            print(f"  {col}: {metrics[col]}")
        print("----------------------------")
    return general_stats


In [ ]:
train_df["sql_tags"].value_counts()

sql_tags
HAVING      15
WHERE        8
ORDERBY      3
SELECT       3
GROUPBY      2
SIMPLE       2
DISTINCT     1
Name: count, dtype: int64

In [ ]:
valid_df["sql_tags"].value_counts()

sql_tags
HAVING      19
WHERE        9
SELECT       4
GROUPBY      3
ORDERBY      3
SIMPLE       3
DISTINCT     1
Name: count, dtype: int64

In [ ]:
test_df["sql_tags"].value_counts()

sql_tags
HAVING      18
WHERE       10
SELECT       4
GROUPBY      3
ORDERBY      3
SIMPLE       3
DISTINCT     2
Name: count, dtype: int64

In [ ]:
valid_df_stats=calculate_and_log_stats(None,None,valid_df,valid=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors
Evaluating QA tests: 100%|██████████| 42/42 [00:00<00:00, 245.61it/s]


General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.36904761904761907
general_avg_cell_recall: 0.36904761904761907
general_avg_tuple_cardinality: 0.24764285714285714
general_avg_tuple_constraint: 0.11904761904761904
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: DISTINCT
  cell_precision_predictions_TAPAS: 0.0
  cell_recall_predictions_TAPAS: 0.0
  tuple_cardinality_predictions_TAPAS: 0.0
  tuple_constraint_predictions_TAPAS: 0.0
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: GROUPBY
  cell_precision_predictions_TAPAS: 0.3333333333333333
  cell_recall_predictions_TAPAS: 0.3333333333333333
  tuple_cardinality_predictions_TAPAS: 0.3333333333333333
  tuple_constraint_predictions_TAPAS: 0.0
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.7105263157894737
  cell_recall_predictions_TAPAS: 0.7105263157894737
  tuple_cardinality_predicti

In [ ]:
valid_stats_df = pd.DataFrame([valid_df_stats])
valid_stats_df.to_csv('/content/drive/MyDrive/TAPEX_Ressources/Results/valid_stats_df_medicine_80prct.csv', index=False)

In [ ]:
test_df_stats=calculate_and_log_stats(None,None,test_df,valid=False)

Evaluating QA tests: 100%|██████████| 43/43 [00:00<00:00, 237.26it/s]


General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.4147209302325581
general_avg_cell_recall: 0.408906976744186
general_avg_tuple_cardinality: 0.22597674418604652
general_avg_tuple_constraint: 0.06976744186046512
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: DISTINCT
  cell_precision_predictions_TAPAS: 0.5
  cell_recall_predictions_TAPAS: 0.5
  tuple_cardinality_predictions_TAPAS: 0.0665
  tuple_constraint_predictions_TAPAS: 0.0
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: GROUPBY
  cell_precision_predictions_TAPAS: 0.3333333333333333
  cell_recall_predictions_TAPAS: 0.25
  tuple_cardinality_predictions_TAPAS: 0.22233333333333336
  tuple_constraint_predictions_TAPAS: 0.0
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.75
  cell_recall_predictions_TAPAS: 0.75
  tuple_cardinality_predictions_TAPAS: 0.37683333333333335
  tuple_co

In [ ]:
test_stats_df = pd.DataFrame([test_df_stats])
test_stats_df.to_csv('/content/drive/MyDrive/TAPEX_Ressources/Results/test_stats_df_medicine_80prct.csv', index=False)

In [ ]:
spider_test_stats=calculate_and_log_stats(None,None,spider_test_pd,valid=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1045 > 1024). Running this sequence through the model will result in indexing errors
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1045. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1043. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1044. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 553 was too long and was skipped.
The input for row 559 was too long and was skipped.
The input for row 560 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1034. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1035. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1036. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1039. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 577 was too long and was skipped.
The input for row 578 was too long and was skipped.
The input for row 579 was too long and was skipped.
The input for row 580 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1142. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1143. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1086. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1085. the input will be skipped
  warnings.warn(f'After tokenization'


...

The input for row 984 was too long and was skipped.
The input for row 985 was too long and was skipped.
The input for row 986 was too long and was skipped.
The input for row 987 was too long and was skipped.
The input for row 988 was too long and was skipped.
The input for row 989 was too long and was skipped.
The input for row 994 was too long and was skipped.
The input for row 995 was too long and was skipped.
The input for row 996 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1084. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 997 was too long and was skipped.


Evaluating QA tests: 100%|██████████| 211/211 [00:04<00:00, 45.19it/s]

General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.5442322274881516
general_avg_cell_recall: 0.5457345971563982
general_avg_tuple_cardinality: 0.49359241706161117
general_avg_tuple_constraint: 0.287521327014218
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.3349166666666667
  cell_recall_predictions_TAPAS: 0.5625
  tuple_cardinality_predictions_TAPAS: 0.45733333333333337
  tuple_constraint_predictions_TAPAS: 0.38891666666666663
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: ORDERBY
  cell_precision_predictions_TAPAS: 0.739361111111111
  cell_recall_predictions_TAPAS: 0.6658055555555557
  tuple_cardinality_predictions_TAPAS: 0.2010833333333334
  tuple_constraint_predictions_TAPAS: 0.1111111111111111
  tuple_order_predictions_TAPAS: 0.3763888888888889
----------------------------
SQL Tag: SELECT
  cell_precision_predictions_TAPAS: 0.31372
  cell_recall_pred

In [ ]:
spider_test_stats=calculate_and_log_stats(None,None,spider_test_pd,valid=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1045 > 1024). Running this sequence through the model will result in indexing errors
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1045. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1043. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1044. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 553 was too long and was skipped.
The input for row 559 was too long and was skipped.
The input for row 560 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1034. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1035. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1036. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1039. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 577 was too long and was skipped.
The input for row 578 was too long and was skipped.
The input for row 579 was too long and was skipped.
The input for row 580 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1142. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1143. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1086. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1085. the input will be skipped
  warnings.warn(f'After tokenization'


...

The input for row 984 was too long and was skipped.
The input for row 985 was too long and was skipped.
The input for row 986 was too long and was skipped.
The input for row 987 was too long and was skipped.
The input for row 988 was too long and was skipped.
The input for row 989 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1135. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1135. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1084. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1084. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 994 was too long and was skipped.
The input for row 995 was too long and was skipped.
The input for row 996 was too long and was skipped.
The input for row 997 was too long and was skipped.


Evaluating QA tests: 100%|██████████| 211/211 [00:05<00:00, 40.33it/s]

General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.5486303317535545
general_avg_cell_recall: 0.552872037914692
general_avg_tuple_cardinality: 0.5028957345971562
general_avg_tuple_constraint: 0.29120379146919434
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.3205
  cell_recall_predictions_TAPAS: 0.48150000000000004
  tuple_cardinality_predictions_TAPAS: 0.458
  tuple_constraint_predictions_TAPAS: 0.287
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: ORDERBY
  cell_precision_predictions_TAPAS: 0.714111111111111
  cell_recall_predictions_TAPAS: 0.6596388888888891
  tuple_cardinality_predictions_TAPAS: 0.19800000000000006
  tuple_constraint_predictions_TAPAS: 0.1111111111111111
  tuple_order_predictions_TAPAS: 0.3819444444444444
----------------------------
SQL Tag: SELECT
  cell_precision_predictions_TAPAS: 0.23751999999999998
  cell_recall_predictions_TAPAS:

In [ ]:
spider_test_stats=calculate_and_log_stats(None,None,spider_test_pd,valid=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1045 > 1024). Running this sequence through the model will result in indexing errors
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1045. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1043. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1044. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 553 was too long and was skipped.
The input for row 559 was too long and was skipped.
The input for row 560 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1034. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1035. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1036. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1039. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 577 was too long and was skipped.
The input for row 578 was too long and was skipped.
The input for row 579 was too long and was skipped.
The input for row 580 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1142. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1143. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1086. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1085. the input will be skipped
  warnings.warn(f'After tokenization'


...

The input for row 984 was too long and was skipped.
The input for row 985 was too long and was skipped.
The input for row 986 was too long and was skipped.
The input for row 987 was too long and was skipped.
The input for row 988 was too long and was skipped.
The input for row 989 was too long and was skipped.
The input for row 994 was too long and was skipped.
The input for row 995 was too long and was skipped.
The input for row 996 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1084. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 997 was too long and was skipped.


Evaluating QA tests: 100%|██████████| 211/211 [00:00<00:00, 213.62it/s]

General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.5502417061611373
general_avg_cell_recall: 0.5545355450236967
general_avg_tuple_cardinality: 0.48390995260663494
general_avg_tuple_constraint: 0.2906777251184834
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.3114166666666667
  cell_recall_predictions_TAPAS: 0.48608333333333337
  tuple_cardinality_predictions_TAPAS: 0.48808333333333337
  tuple_constraint_predictions_TAPAS: 0.27775
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: ORDERBY
  cell_precision_predictions_TAPAS: 0.7129444444444444
  cell_recall_predictions_TAPAS: 0.6655277777777778
  tuple_cardinality_predictions_TAPAS: 0.19800000000000006
  tuple_constraint_predictions_TAPAS: 0.1111111111111111
  tuple_order_predictions_TAPAS: 0.37777777777777777
----------------------------
SQL Tag: SELECT
  cell_precision_predictions_TAPAS: 0.2
  cell_recall_pre

In [ ]:
spider_test_stats=calculate_and_log_stats(None,None,spider_test_pd,valid=False)

<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1045. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1043. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1044. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 553 was too long and was skipped.
The input for row 559 was too long and was skipped.
The input for row 560 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1034. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1035. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1036. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1039. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 577 was too long and was skipped.
The input for row 578 was too long and was skipped.
The input for row 579 was too long and was skipped.
The input for row 580 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1142. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1143. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1086. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1085. the input will be skipped
  warnings.warn(f'After tokenization'


...

The input for row 984 was too long and was skipped.
The input for row 985 was too long and was skipped.
The input for row 986 was too long and was skipped.
The input for row 987 was too long and was skipped.
The input for row 988 was too long and was skipped.
The input for row 989 was too long and was skipped.
The input for row 994 was too long and was skipped.
The input for row 995 was too long and was skipped.
The input for row 996 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1084. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 997 was too long and was skipped.


Evaluating QA tests: 100%|██████████| 211/211 [00:01<00:00, 181.56it/s]

General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.5502417061611373
general_avg_cell_recall: 0.5545355450236967
general_avg_tuple_cardinality: 0.48390995260663494
general_avg_tuple_constraint: 0.2906777251184834
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.3114166666666667
  cell_recall_predictions_TAPAS: 0.48608333333333337
  tuple_cardinality_predictions_TAPAS: 0.48808333333333337
  tuple_constraint_predictions_TAPAS: 0.27775
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: ORDERBY
  cell_precision_predictions_TAPAS: 0.7129444444444444
  cell_recall_predictions_TAPAS: 0.6655277777777778
  tuple_cardinality_predictions_TAPAS: 0.19800000000000006
  tuple_constraint_predictions_TAPAS: 0.1111111111111111
  tuple_order_predictions_TAPAS: 0.37777777777777777
----------------------------
SQL Tag: SELECT
  cell_precision_predictions_TAPAS: 0.2
  cell_recall_pre

In [ ]:
spider_test_stats=calculate_and_log_stats(None,None,spider_test_pd,valid=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1045 > 1024). Running this sequence through the model will result in indexing errors
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1045. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1043. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1044. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 553 was too long and was skipped.
The input for row 559 was too long and was skipped.
The input for row 560 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1034. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1035. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1036. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1039. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 577 was too long and was skipped.
The input for row 578 was too long and was skipped.
The input for row 579 was too long and was skipped.
The input for row 580 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1142. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1143. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1086. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1085. the input will be skipped
  warnings.warn(f'After tokenization'


...

The input for row 984 was too long and was skipped.
The input for row 985 was too long and was skipped.
The input for row 986 was too long and was skipped.
The input for row 987 was too long and was skipped.
The input for row 988 was too long and was skipped.
The input for row 989 was too long and was skipped.


<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1135. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1135. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1084. the input will be skipped
  warnings.warn(f'After tokenization'
<ipython-input-18-d380f16ec0a8>:44: UserWarning: After tokenization the input is longer than 1024 tokens: 1084. the input will be skipped
  warnings.warn(f'After tokenization'


The input for row 994 was too long and was skipped.
The input for row 995 was too long and was skipped.
The input for row 996 was too long and was skipped.
The input for row 997 was too long and was skipped.


Evaluating QA tests: 100%|██████████| 211/211 [00:09<00:00, 23.35it/s]

General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.5216066350710901
general_avg_cell_recall: 0.5181990521327013
general_avg_tuple_cardinality: 0.46490521327014206
general_avg_tuple_constraint: 0.265085308056872
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.20833333333333334
  cell_recall_predictions_TAPAS: 0.3333333333333333
  tuple_cardinality_predictions_TAPAS: 0.48299999999999993
  tuple_constraint_predictions_TAPAS: 0.16666666666666666
  tuple_order_predictions_TAPAS: 0.0
----------------------------
SQL Tag: ORDERBY
  cell_precision_predictions_TAPAS: 0.6855
  cell_recall_predictions_TAPAS: 0.61875
  tuple_cardinality_predictions_TAPAS: 0.15911111111111115
  tuple_constraint_predictions_TAPAS: 0.07222222222222223
  tuple_order_predictions_TAPAS: 0.3777777777777778
----------------------------
SQL Tag: SELECT
  cell_precision_predictions_TAPAS: 0.14364000000000002
  cell_recall

In [ ]:
spider_stats_df = pd.DataFrame([spider_test_stats])
spider_stats_df.to_csv('/content/drive/MyDrive/TAPEX_Ressources/Results/train_df_spider_100prct_Metrics_June_Corrected.csv', index=False)